# Exploratory Data Analysis

In [ ]:
import os, sys, import_ipynb, ipynb
if os.getcwd() == 'C:\\Users\\admin\\Desktop\\retail_data_analysis\\code':
    print ("Already in code directory")
else:
    os.chdir('..//code')
        
# import python libraries
from importLibraries import *


def fun_eda():
    os.chdir(os.path.expanduser('../input'))
    input_data_preProcessed_chunk = pd.read_csv('input_data_preProcessed.csv', chunksize=10000, iterator=True, header=0, skipinitialspace=True, index_col=False)  
    input_data_preProcessed = pd.concat(input_data_preProcessed_chunk, ignore_index=True)
    
    # convert created_at to datetime object
    input_data_preProcessed['new_Only_InvoiceDate'] = pd.to_datetime(input_data_preProcessed['new_Only_InvoiceDate'], format='%Y-%m-%d %H:%M:%S.%f')
    input_data_preProcessed['new_Only_InvoiceDate'] = input_data_preProcessed['new_Only_InvoiceDate'].dt.date
    
    # convert UnitPrice column from int64 to object
    #input_data_preProcessed['UnitPrice'] = input_data_preProcessed['UnitPrice'].apply(lambda x: str(x))
    
    #save plots
    os.chdir('..//output')
    
    # EDA of the Description column
    tmpdf = input_data_preProcessed[['Description']]
    display(tmpdf[['Description']].head())
    # Count of stopwords
    stop = stopwords.words('english')
    tmpdf['stopwords'] = tmpdf['Description'].apply(lambda x: len([x for x in x.split() if x in stop]))
    display(tmpdf[['Description','stopwords']].head())
    # Count of special characters 
    tmpdf['specialChars'] = tmpdf['Description'].apply(lambda x: len([x for x in x.split() if x.count("-")]))
    display(tmpdf[['Description','specialChars']].head())
    # Remove stopwords
    stop = stopwords.words('english')
    tmpdf['Description'] = tmpdf['Description'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    display(tmpdf['Description'].head())
    # Remove commonly occuring words
    freq = pd.Series(' '.join(tmpdf['Description']).split()).value_counts()[:5]
    freq = list(freq.index)
    tmpdf['Description'] = tmpdf['Description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
    display(tmpdf['Description'].head())
    # Rare words removal
    freq = pd.Series(' '.join(tmpdf['Description']).split()).value_counts()[-10:]
    freq = list(freq.index)
    tmpdf['Description'] = tmpdf['Description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
    display(tmpdf['Description'].head())
    
    
    # Countvec for correlation
    tmpdf = input_data_preProcessed[['Description']]
    #tmpdf = tmpdf.head(20)
    tmpdf['Description'] = tmpdf['Description'].apply(lambda x: "".join(x) )
    tmpdf = np.array(tmpdf['Description'].tolist())
    tf_vectorizer = CountVectorizer(max_df=1, min_df=0, analyzer='word', ngram_range=(1, 2), max_features=1000, stop_words='english')
    tf = tf_vectorizer.fit_transform(tmpdf)
    #display(tf.toarray())
    # Get feature names
    feature_names = tf_vectorizer.get_feature_names()
    # View feature names
    print(feature_names)
    tf_as_df = pd.DataFrame(tf.toarray(), columns=feature_names)
    #display(tf_as_df)
    corr = tf_as_df.corr()
    sns.heatmap(corr, annot=True)
    #tf_as_df = tf_as_df.sparse.to_dense()
    #display(tf_as_df)
    
    
    # Plot 
    fig = plt.figure(figsize=(5, 5))
    #tmpdf = input_data_preProcessed[(input_data_preProcessed.new_Only_InvoiceDate >= "2010-01-01") & (input_data_preProcessed.new_Only_InvoiceDate <= "2011-02-28")]
    tmpdf = input_data_preProcessed.groupby([input_data_preProcessed['Country']])['revenue'].sum().reset_index(name="sum_revenue")
    chart = sns.barplot(x="Country", y="sum_revenue", data=tmpdf)
    plt.setp(chart.get_xticklabels(), rotation=90)
    plt.xlabel('Country')
    plt.ylabel('sum_revenue')
    plt.title('Country vs sum_revenue')
    plt.show
    plt.savefig('sum_revenue_vs_Country.jpg')
    
    
    # Plot 
    fig = plt.figure(figsize=(5, 5))
    #tmpdf = input_data_preProcessed[(input_data_preProcessed.new_Only_InvoiceDate >= "2010-01-01") & (input_data_preProcessed.new_Only_InvoiceDate <= "2011-02-28")]    
    tmpdf = input_data_preProcessed.groupby(['new_Only_InvoiceDate'])['Quantity'].agg('sum').reset_index().rename(columns={'Quantity':'SumQuantity'})
    fig, ax1 = plt.subplots(figsize = (10,10))    
    fig = sns.barplot(x = "new_Only_InvoiceDate", y = "SumQuantity", data = tmpdf, ci = None, ax=ax1)
    plt.setp(fig.get_xticklabels(), rotation=90)
    plt.xlabel('new_Only_InvoiceDate')
    plt.ylabel('Quantity')
    plt.title('new_Only_InvoiceDate vs Quantity')
    plt.show
    plt.savefig('new_Only_InvoiceDate_vs_SumQuantity.jpg')
    
    
    
    # Plot two figures in the two axes of the same row
    fig, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2, figsize=(10,10))
    fig.suptitle('Price, Revenue, and Quantity')
    #tmpdf = input_data_preProcessed[(input_data_preProcessed.new_Only_InvoiceDate >="2010-01-01") & (input_data_preProcessed.new_Only_InvoiceDate <= "2011-02-28")]
    chart = sns.lineplot(x="Quantity", y="revenue", data=input_data_preProcessed, ax=ax1)
    plt.xlabel('Quantity')
    plt.ylabel('revenue')
    #plt.title('Quantity vs revenue')
    
    #tmpdf = input_data_preProcessed[(input_data_preProcessed.new_Only_InvoiceDate >= "2010-01-01") & (input_data_preProcessed.new_Only_InvoiceDate <= "2011-02-28")]
    chart = sns.lineplot(x="UnitPrice", y="revenue", data=input_data_preProcessed, ax=ax2)
    plt.setp(chart.get_xticklabels(), rotation=90)
    plt.xlabel('UnitPrice')
    plt.ylabel('revenue')
    #plt.title('UnitPrice vs revenue')
    plt.show
    plt.savefig('UnitPrice_vs_revenue_vs_Quantity.jpg')
    
    
    # Plot twin axis
    #tmpdf = input_data_preProcessed[(input_data_preProcessed.new_Only_InvoiceDate >= "2010-01-01") & (input_data_preProcessed.new_Only_InvoiceDate <= "2011-02-28")]
    tmpdf = input_data_preProcessed.groupby([input_data_preProcessed['UnitPrice']])['Quantity', 'revenue'].sum().reset_index().rename(columns={'Quantity':'SumQuantity','revenue':'SumRevenue'})
    fig,ax1 = plt.subplots(figsize=(10,10))
    plt.xticks(rotation=90)
    ax1.plot(tmpdf.UnitPrice, tmpdf.SumQuantity, color="red", alpha=0.5)
    ax1.set_xlabel("UnitPrice",fontsize=14)
    ax1.set_ylabel("SumQuantity",color="red",fontsize=14)
    # second axis
    ax2=ax1.twinx()
    ax2.plot(tmpdf.UnitPrice, tmpdf.SumRevenue, color="yellow", alpha=0.5)
    ax2.set_ylabel("SumRevenue",color="yellow",fontsize=14)
    plt.show
    fig.savefig('QuantityVsRevenueVsPrice.jpg', format='jpeg', dpi=100, bbox_inches='tight')
    
    
    